This notebooks uses the following dataset <br>
https://archive.ics.uci.edu/ml/datasets/Motion+Capture+Hand+Postures

# 1. Analysis of the Problem 

## Understand the problem

working hypothesis
think about model architecture
loss function
and loss criteria

select a measure of success
    Detect all classes/types of hand gestures with a high accuracy
    Classification Problem
    Check whether class-imbalanced problems

Dataset with class of the hand gesture/'Class' (our target value) the person which performed the hand gesture ('User') and a 
feature vector that consists 11 subvectors. Each subvector contains X, Y and Z coordinates. Those coordinates belong to one of the detected markes on the hand glove the user is wearing

* motion capture camera records 12 users performing 5 hand postures with markers attached to a left-handed glove
* rigid pattern of markers on the back of the glove -> establish local coordinate system for the hand
* 11 markers were attached to the thumb and fingers of the glove
* there is no a priori correspondence between the markers of two given records
* due to the resolution of the capture volume and self-occlusion due to the orientation and configuration of the hand and fingers, many records have missing markers.
    -> the number of visible markers in a record varied considerably.


The Problem:
We cannot easily apply traditional approaches because of two properties of point clouds:
* unordered collection (Point 1 with X,Y and Z coordinates could refer to the thh )
* the size of the point cloud varies (due to occlusion, etc.)

## Useful Information from the authors/paper

Class label:<br>
1=Fist(with thumb out), 2=Stop(hand flat), 3=Point1(point with pointer finger), 4=Point2(point with pointer and middle fingers), 5=Grab(fingers curled as if to grab)


Preprocessing:<br>
all markers were transformed to the local coordinate system of the record containing them.

Reduce number of records:<br>
each transformed marker with a norm greater than 200 millimeters was pruned. 
records that contained fewer than 3 markers was removed. 
the data has at most 12 markers per record and at least 3

Be careful:<br>
It is likely that for a given record and user there exists a near duplicate record originating from the same user.
-> evaluate on leave-one-user-out basis wherein

## Ideas I want to try out


**My goal**

* able to predict which gesture a person is performing

* achieve a high accuracy

**My idea/approach**

* 1.approach: extract features that are meaningful for a given data point, train conventional models

* 2.approach: use models that are adapted to point clouds and have been developed specifically for this data

# 2. Data Exploration and Preparation

Check for missing values, NaN values or features,
uniqueness of the data (is it as it was expected to be)
understand the variations in the data (statistical tools)
outliers?
Is it possible to combine features?
Do you have unnecessary features? (a column which gives no information – for instance name
column– or a feature you consider unrelated to the problem)
Check the correlation matrix. It will tell you how much the features are related. You may say,
for instance, there is a great potential to reduce number of dimensions.
scale data (for example, in the [-1, 1] range).

## Preparing the enviroment

In [ ]:
#%reload_ext autoreload
#%autoreload 2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 60
import time
import pickle
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import keras_tuner
from tensorflow.keras import layers
from typing import Tuple, Optional, Callable
from sklearn.decomposition import PCA
import plotly.express as px
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# My custom functions
from scripts import analyze_helper, visualisation

# Utility functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, precision_recall_curve, average_precision_score, recall_score, f1_score, accuracy_score, precision_score
from sklearn import preprocessing as pp
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.utils import to_categorical
from os.path import dirname, abspath, join
from tqdm.keras import TqdmCallback
from random import shuffle

# Models we want to use
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, PredefinedSplit

## Write utility functions I want to use multiple times

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_cm(y_true, y_pred):

    plt.clf()

    class_labels = ['Fist', 'Stop', 'Point1', 'Point2', 'Grab']
    cf_matrix = confusion_matrix(y_true, y_pred, labels=[1, 2, 3, 4, 5])
    
    group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
    #normalize
    cf_matrix = (cf_matrix.T/cf_matrix.sum(axis=1)).T
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten()]
    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(5,5)

    custom_cmap = sns.light_palette("#009682", as_cmap=True)
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap=custom_cmap, xticklabels=class_labels, yticklabels=class_labels, cbar=False)
    plt.ylabel('true label')
    plt.xlabel('predicted label')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.show()

In [ ]:
class Report:

    report: dict

    def __init__(self, model_name:str, model, X_test, y_test, description:list=[]):
        self.X_test = X_test
        self.y_test = y_test
        self.description = description
        self.model_name = model_name

        self.report = self.create_report(model)


    def create_report(self, model):

        try:
            if type(model) == keras.Sequential:
                y_score = model.predict(self.X_test)
            else:
                y_score = model.predict_proba(self.X_test)
        except AttributeError:
            print('No report possible, because no predict_proba method')
            report = None
            return
        y_score_2 = model.predict(self.X_test)

        if type(model) != keras.Sequential:
            report = classification_report(self.y_test, y_score_2, output_dict=True)
        else:
            y_score_2 = to_categorical(np.argmax(y_score_2, axis=1))
            report = classification_report(self.y_test, y_score_2, output_dict=True)
            #NN report does not have accuracy score -> have a look why
            accuracy = accuracy_score(self.y_test, y_score_2)
            report['accuracy'] = accuracy

        return report
    
    def get_report_as_df(self, df_to_save_to:pd.DataFrame):
        'Get the report as a dataframe'

        new_row = {'model': self.model_name,
                   'Accuracy': round(self.report["accuracy"],6), 
                   'Precision': round(self.report["macro avg"]["precision"],5),
                   'Recall': round(self.report["macro avg"]["recall"],6),
                   'F1_score': round(self.report["macro avg"]["f1-score"],6)} 
        df_to_save_to = df_to_save_to.append(new_row, ignore_index=True)

        return df_to_save_to


    def __str__(self):

        if self.report is None:
            print('No return possbile')
            return 'No valid return'

        pattern = '''
        ***********************REPORT*******************************
        Average (macro) precision: {}
        Average accuracy: {}
        Average (macro) recall: {}
        Average (macro) f1-score: {}
        Description {}
        ************************************************************
        '''
        return pattern.format(round(self.report["macro avg"]["precision"],5), round(self.report["accuracy"],6), round(self.report["macro avg"]["recall"],6), round(self.report["macro avg"]["f1-score"],6), ', '.join(self.description))

In [ ]:
def save_eval_dict_pkl(eval_dict, name):

    # Get path to save the history
    eval_folder = 'model/single_run/eval'
    model_path = join(eval_folder, name)
    
    with open(model_path, 'wb') as file_pi:
        pickle.dump(eval_dict, file_pi)

def load_eval_dict_pkl(name) -> dict:

    # Get path to save the history
    eval_folder = 'model/single_run/eval'
    model_path = join(eval_folder, name)

    with open(model_path, 'rb') as file_pi:
        eval_dict = pickle.load(file_pi)
    
    return eval_dict

def save_to_eval_dict(eval_dict:dict, split_set:str, acc:float, precission:float, recall:float, f1:float):
    ''' Save multiple metrics to a dictionary for evaluation '''

    if split_set not in ['train', 'test']:
        raise ValueError('split_set must be either train or test')
    
    eval_dict[split_set]['accuracy'].append(acc)
    eval_dict[split_set]['precision'].append(precission)
    eval_dict[split_set]['recall'].append(recall)
    eval_dict[split_set]['f1'].append(f1)

    return eval_dict

In [ ]:
def plot_eval_dict_barplot_new(data):

    # Extracting data
    train_acc = data['train']['accuracy']
    train_prec = data['train']['precision']
    train_recall = data['train']['recall']
    train_f1 = data['train']['f1']
    test_acc = data['test']['accuracy']
    test_prec = data['test']['precision']
    test_recall = data['test']['recall']
    test_f1 = data['test']['f1']

    # Creating subplots for accuracy and precision
    fig, axs = plt.subplots(2, 2, figsize=(15, 5))
    bar_width = 0.2

    # Plot for accuracy
    axs[0][0].bar([i+bar_width/2 for i in range(len(train_acc))], train_acc, width=bar_width, color='b', label='Train Accuracy')
    axs[0][0].bar([i-bar_width/2 for i in range(len(test_acc))], test_acc, width=bar_width, color='r', label='Test Accuracy')
    axs[0][0].set_xticks(range(len(train_acc)))
    axs[0][0].set_xticklabels([f'CV{i}' for i in range(len(train_acc))])
    axs[0][0].set_ylabel('Accuracy')
    axs[0][0].set_title('Accuracy')
    axs[0][0].legend()

    # Plot for precision
    axs[0][1].bar([i+bar_width/2 for i in range(len(train_prec))], train_prec, width=bar_width, color='b', label='Train Precision')
    axs[0][1].bar([i-bar_width/2 for i in range(len(test_prec))], test_prec, width=bar_width, color='r', label='Test Precision')
    axs[0][1].set_xticks(range(len(train_acc)))
    axs[0][1].set_xticklabels([f'CV{i}' for i in range(len(train_acc))])
    axs[0][1].set_ylabel('Precision')
    axs[0][1].set_title('Precision')
    axs[0][1].legend()

    # Plot for recall
    axs[1][0].bar([i+bar_width/2 for i in range(len(train_recall))], train_recall, width=bar_width, color='b', label='Train Recall')
    axs[1][0].bar([i-bar_width/2 for i in range(len(test_recall))], test_recall, width=bar_width, color='r', label='Test Recall')
    axs[1][0].set_xticks(range(len(train_recall)))
    axs[1][0].set_xticklabels([f'CV{i}' for i in range(len(train_recall))])
    axs[1][0].set_ylabel('Recall')
    axs[1][0].set_title('Recall')
    axs[1][0].legend()

    # Plot for f1_score
    axs[1][1].bar([i+bar_width/2 for i in range(len(train_f1))], train_f1, width=bar_width, color='b', label='Train F1-Score')
    axs[1][1].bar([i-bar_width/2 for i in range(len(test_f1))], test_f1, width=bar_width, color='r', label='Test F1-Score')
    axs[1][1].set_xticks(range(len(train_f1)))
    axs[1][1].set_xticklabels([f'CV{i}' for i in range(len(train_f1))])
    axs[1][1].set_ylabel('F1 Score')
    axs[1][1].set_title('F1 Score')
    axs[1][1].legend()

    plt.show()

## Loading the Data

In [ ]:
file_path_raw = os.path.join('data', 'Postures.csv')
df_raw = pd.read_csv(file_path_raw, sep=',', na_values='?')

## Understand the Data

### Choose a metric

To choose a metric we have to check whether the classes are balenced/unbalanced

In [ ]:
class_counts = df_raw['Class'].value_counts(sort=False)
class_counts.plot(kind='bar', title='Gestures Class Distibution')
plt.xlim(0.5, df_raw['Class'].max()+0.5)
plt.show()

To jugde the model perfomance we will choose a mixture of multiple metrics:
* Confusion Matrix
* Accuracy (we have balanced classes!)
* Recall (made TP predictions divided by possible TP predictions)
* precission-recall plot
* F1-Score (harmonic mean between recall and precision, combines the two metrics into one value)

TODO 
Maybe Later
* ROC AUC (Receiver Operator Characteristic — Area Under the Curve, we want a high TPR with a low FPR)
Note: we will use the One vs Rest strategy for ROC AUC (because it is usally a metric for binary classification)

Write utility function to use the ROC AUC for multiclass classficiation

In [ ]:
def precision_recall_multiclass(model, X_test, y_test):

    plt.clf()

    y_score = model.predict_proba(X_test)
    y_score_2 = model.predict(X_test)
    
    #print('Amount and Distribution of Test Data: \n', y_test.value_counts())
    y_bin = pp.label_binarize(y_test, classes=model.classes_)

    precision = dict()
    recall = dict()
    average_precision = dict()
    classes = model.classes_

    for i in range(len(classes)):
        precision[i], recall[i], _ = precision_recall_curve(y_bin[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_bin[:, i], y_score[:, i])
        #print('\n average precision: ', classes[i], ': ', average_precision[i])
        try:
            plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(classes[i]))
        except Exception as e:
            print(e)

    
    plt.xlabel("recall")
    plt.ylabel("precision")
    plt.legend(loc="best")
    plt.title("precision vs. recall curve")
    plt.show()

## Data Exploration

In [ ]:
df_raw.columns

In [ ]:
# Check which columns have missing values
print(f"Missing values in: {analyze_helper.check_for_missing_vals(df_raw)}")
# Compute missing ratio, hide columns with no missing values (0.0%)
analyze_helper.compute_missing_ratio(df_raw)

Hint 1 for preprocessing:
drop the coordinates (X,Y,Z) for point 10 and 11

In [ ]:
df_raw = df_raw.fillna(0)

In [ ]:
df_raw.head(5)
df_raw.drop([0], inplace=True)
df_raw.reset_index(drop=True, inplace=True)

In [ ]:
print('Number of Instances  : ', df_raw.shape[0])
print('Number of Attributes : ', df_raw.shape[1])
print('Number of target classes   : ', df_raw['Class'].nunique())
print('Number of users   : ', df_raw['User'].nunique())

In [ ]:
df_raw['User'].unique()

The description says the data set contains 12 User. No information provided (why are 14 user in the data?).
Hint for us to drop User 4 and 7 ?
They both have signifiantly less data points

In [ ]:
user_group = df_raw.groupby(['User'], sort=False)
user_group.count()

In [ ]:
df_raw.info()

### Data Visualization

In [ ]:
def pltHand(handPoints):
    plt.close('all')
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    
    for i in range(11):
        pntx = f'X{i}'
        pnty = f'Y{i}'
        pntz = f'Z{i}'
        
        if(handPoints[pntx].values[0] == 0 or
            handPoints[pnty].values[0] == 0 or
            handPoints[pntz].values[0] == 0):
            n = 0;
        else:
            xlocation = handPoints[pntx]
            ylocation = handPoints[pnty]
            zlocation = handPoints[pntz]
            ax.scatter(xlocation, ylocation, zlocation, marker='v')
    
    crntClass = handPoints['Class'].values[0]
    if (crntClass == 1):
        title = 'Fist + Thumb out'
    if(crntClass == 2):
        title = 'Stop/Flat hand'
    if (crntClass == 3):
        title = 'Point with pointer finger'
    if (crntClass == 4):
        title = 'Point with pointer + middle finger'
    if (crntClass == 5):
        title = 'Grab'
    
    plt.title(title)
    plt.show()


In [ ]:
def scatterPlot2D(x_DF, y_DF, labels:list):
    #df = pd.DataFrame(data=x_DF.iloc[:,0:2], index=x_DF.index)
    df = pd.DataFrame(data=x_DF, index=x_DF.index)
    df = pd.concat((df,y_DF), axis=1, join="inner")
    df.columns = ["First Dimension", "Second Dimension", "Class"]
    sns.lmplot(x="First Dimension",y="Second Dimension", hue="Class", data=df, fit_reg=False)
    
    ax = plt.gca()
    ax.set_title("Visualization of the data segragation using ")

def scatterPlot3D(x_DF, y_DF, labels:dict):
    df = pd.DataFrame(data=x_DF.iloc[:,0:3], index=x_DF.index)
    df = pd.concat((df,y_DF), axis=1, join="inner")
    fig = px.scatter_3d(df, x=df.columns[0], y=df.columns[1], z=df.columns[2],\
                          color='Class', symbol='Class', opacity=0.7, \
                          color_continuous_scale=px.colors.sequential.Viridis, width = 600, height = 500,
                       labels=labels) 

    title = "Visualization of the data segragation using "
    fig.update_layout(title_text=title, showlegend = True, hovermode = False)
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01))

    fig.show() 

In [ ]:
# Plot a random hand gesture from the dataset to get a an idea of the data
from mpl_toolkits.mplot3d import Axes3D 

for _ in range(4):
    rand_dp = np.random.randint(df_raw.shape[0], size=1)[0]
    pltHand(df_raw[rand_dp:rand_dp+1] )


In [ ]:
def correlation_matrix(df: pd.DataFrame):

    correlationMatrix = pd.DataFrame(df_raw).corr() 
    f = plt.figure(figsize=(12, 6))
    plt.matshow(correlationMatrix, fignum=f.number)
    plt.xticks(range(df_raw.shape[1]), df_raw.columns, fontsize=14, rotation=75)
    plt.yticks(range(df_raw.shape[1]), df_raw.columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.show()

In [ ]:
correlation_matrix(df_raw)

In [ ]:
# Dataframe we want to store x_mean, y_mean and z_mean in
df_xyz = pd.DataFrame()

# We dont want to change the original data set
df_raw_dummy = df_raw.copy(deep=True)

# Save the user and class column
df_user = df_raw_dummy.pop('User')
df_class = df_raw_dummy.pop('Class')

# We replace all zeros with NaN, so the mean calculatiion ignores occluded data points
df_raw_dummy = df_raw_dummy.replace(0, np.NaN)

# Extract the X, Y and Z columns
df_x_vals = df_raw_dummy[df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith('X')]]
df_y_vals = df_raw_dummy[df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith('Y')]]
df_z_vals = df_raw_dummy[df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith('Z')]]

# Extract the mean of the X, Y and Z columns
for coordinate in ['X', 'Y', 'Z']:
    coordinate_cols = df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith(coordinate)]
    df_xyz[f'{coordinate}_mean'] = df_raw_dummy[coordinate_cols].mean(axis=1)
    df_xyz[f'{coordinate}_min'] = df_raw_dummy[coordinate_cols].min(axis=1)
    df_xyz[f'{coordinate}_max'] = df_raw_dummy[coordinate_cols].max(axis=1)

# Add user and class label
df_xyz['User'] = df_user
df_xyz['Class'] = df_class
# Change NaN back to 0 values
df_raw_dummy = df_raw_dummy.fillna(0)

In [ ]:
df_xyz

In [ ]:
df_xyz = df_xyz.reset_index(drop=True)

In [ ]:
df_xyz

In [ ]:
class_labels = ['Fist', 'Stop', 'Point1', 'Point2', 'Grab']
df_xyz = df_xyz.reset_index(drop=True)
scatterPlot3D(df_xyz[['X_mean', 'Y_mean', 'Z_mean']], df_class, class_labels)

#### PCA

Principal Component Analysis
* reduce the dimensionality of the data.
* This makes the data faster and easier to process.
* when dimensionality is reduced in 2D or 3D, data of higher dimensions can be visualized in a human-readable graph.<br>

Best case: the 2D and 3D graph can directly give an understanding of how well the data can be classified. If there are already well visible clusters in the 2D graph, it is reasonable to conclude that a classification algorithm can easily detect this pattern<br>

Lets see how our data looks like after dimensionality reduction

In [ ]:
# TODO compute PCA and plot the data

First we right some functions to plot the results

In [ ]:
pca = PCA()

df_pca = df_raw.copy()
df_pca.drop(['User', 'Class'], inplace=True, axis=1)
pca.fit(df_pca)
features_pca = pd.DataFrame(pca.transform(df_pca))
classes = df_raw[['Class']].copy()
print(pca.explained_variance_ratio_)
print("First 2: ", np.round(sum(pca.explained_variance_ratio_[0:1])*100,2),"%")
print("First 3: ", np.round(sum(pca.explained_variance_ratio_[0:2])*100,2),"%")

In [ ]:
features_pca.shape

In [ ]:
class_labels = ['Fist', 'Stop', 'Point1', 'Point2', 'Grab']
scatterPlot2D(features_pca.iloc[:,0:2], classes, class_labels)
scatterPlot3D(features_pca, classes, class_labels)

Results:<br>
In these two representations we can clearly see that clusters of classes are partially formed. However, these overlap, i.e. the interclass distance is relatively small.
In addition, we see that class 2 forms a second, smaller cluster.

Now we can also have a look how much variance of the data each pca component describes

In [ ]:
def plot_explained_variance(pca:PCA):

    pca_variance_ratio = pca.explained_variance_ratio_
    cum_sum_eigenvalues = np.cumsum(pca_variance_ratio)
    print(f'Total explained variance ratio:', sum(pca_variance_ratio))
    
    plt.bar(range(0,len(pca_variance_ratio)), pca_variance_ratio, alpha=0.5, align='center', label='Individual explained variance')
    plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance', color='r')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal component index')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_explained_variance(pca)

In [ ]:
# TODO results of explained ratio

## Pre-processing



### General

In [ ]:
# Drop rows of user 4 and 7
# Because they have significantly less data points
df_raw = df_raw[df_raw['User'] != 4]
df_raw = df_raw[df_raw['User'] != 7]
df_raw.reset_index(drop=True, inplace=True)

In [ ]:
# Drop coordinates of point 10 and 11
# More than 90% of the data is missing
# Search for "Hint 1" for further information
df_raw.drop(['X10', 'Y10', 'Z10', 'X11', 'Y11', 'Z11'], inplace=True, axis=1)

### Preparing Dataset

#### a. Extract features (min, max, mean, etc.) - df_aggregate

Ideas for new features: (inspired from paper)<br>
* number of markers
* mean (per coordinate)
* Eigenvalues and vectors of the points covariance matrix
https://math.stackexchange.com/questions/2842830/why-does-the-eigen-decomposition-of-the-covariance-matrix-of-a-point-cloud-give
* dimensions of the axis-aligned minimum bounding box centered on the mean

Keep in mind that each feature has to aggregate the points in such a way that the result is order invariant!


In [ ]:
# New data set we want to fill step by step
df_aggregate= pd.DataFrame()
# We dont want to change the original data set
df_raw_dummy = df_raw.copy(deep=True)

# Save the user and class column
df_user = df_raw_dummy.pop('User')
df_class = df_raw_dummy.pop('Class')

# We replace all zeros with NaN, so the mean calculatiion ignores occluded data points
df_raw_dummy = df_raw_dummy.replace(0, np.NaN)


In [ ]:
# Extract the X, Y and Z columns
df_x = df_raw_dummy[df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith('X')]]
df_y = df_raw_dummy[df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith('Y')]]
df_z = df_raw_dummy[df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith('Z')]]

In [ ]:
# Extract the mean, min, max etc. of the X, Y and Z columns
for coordinate in ['X', 'Y', 'Z']:
    coordinate_cols = df_raw_dummy.columns[pd.Series(df_raw_dummy.columns).str.startswith(coordinate)]
    df_aggregate[f'{coordinate}_mean'] = df_raw_dummy[coordinate_cols].mean(axis=1)
    df_aggregate[f'{coordinate}_min'] = df_raw_dummy[coordinate_cols].min(axis=1)
    df_aggregate[f'{coordinate}_max'] = df_raw_dummy[coordinate_cols].max(axis=1)

# Change NaN back to 0 values
df_raw_dummy = df_raw_dummy.fillna(0)

In [ ]:
# Extract the number of visible points (not occluded)
df_aggregate['n_points'] = (df_raw_dummy.astype(bool).sum(axis=1))/3
df_aggregate

In [ ]:
df_raw_dummy

My idea:<br>
find the orientation of a given cluster<br>
(https://math.stackexchange.com/questions/2842830/why-does-the-eigen-decomposition-of-the-covariance-matrix-of-a-point-cloud-give)

1. Rearange the dataset (1 point per row with X, Y, Z value)
-> All points per row will be saved in a batch as new sub dataframe
2. Compute the covariance matrix for each sub dataframe
3. Calculate the Eigenvalues and Eigenvectors of the covariance matrix
4. Concat created features to the dataframe


DISCLAIMER: <br>
The following function can take up to 1 min!

In [ ]:

# Assume you have a DataFrame called 'df' with the columns ['X0', 'Y0', 'Z0', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']
# Reorganize the dataframe to have each row as a batch of data
# 'X0', 'Y0', 'Z0',
# 'X1', 'Y1', 'Z1',
# 'X2', 'Y2', 'Z2'

# Create an empty list to store the batches of data
batches = []

# Iterate over the DataFrame and extract the batches of data
for row in range(0, df_raw_dummy.shape[0]):

    col_batch = []
    # Extract a batch of data for the current row
    for col in range(0, df_raw_dummy.shape[1], 3):
        batch = df_raw_dummy.iloc[row, col:col+3]
        # Rename the columns
        batch.index = ['X', 'Y', 'Z']
        #print(f'batch: {batch}')
        col_batch.append(batch)
    # Append the batch to the list
    batches.append(col_batch)

# Concatenate the batches of data under each other
concat_batches = []
for batch in batches:
    concat_batch = pd.concat(batch, axis=1).transpose()
    concat_batches.append(concat_batch)
    # Remove rows with all zeros
    concat_batch = concat_batch[(concat_batch.T != 0).any()]

# Create a dictionary with the eigenvalues and eigenvectors as values
eigen_dict = {'eigenvec_1_1': [],
                'eigenvec_1_2': [],
                'eigenvec_1_3': [],
                'eigenvec_2_1': [],
                'eigenvec_2_2': [],
                'eigenvec_2_3': [],
                'eigenvec_3_1': [],
                'eigenvec_3_2': [],
                'eigenvec_3_3': [],
              'eigenval_1': [],
              'eigenval_2': [],
              'eigenval_3': []}

# Create the DataFrame
eigen_df = pd.DataFrame(eigen_dict)

# Compute the covariance matrix for each batch
for concat_batch in concat_batches:
    # Compute the covariance matrix
    cov_matrix = np.cov(concat_batch, rowvar=False)
    # Compute the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    # Add the eigenvalues and eigenvectors to the DataFrame
    # Store eigenvector 1
    eigen_dict['eigenvec_1_1'].append(eigenvectors[0, 0])
    eigen_dict['eigenvec_1_2'].append(eigenvectors[1, 0])
    eigen_dict['eigenvec_1_3'].append(eigenvectors[2, 0])
    # Store eigenvector 2
    eigen_dict['eigenvec_2_1'].append(eigenvectors[0, 1])
    eigen_dict['eigenvec_2_2'].append(eigenvectors[1, 1])
    eigen_dict['eigenvec_2_3'].append(eigenvectors[2, 1])
    # Store eigenvector 3
    eigen_dict['eigenvec_3_1'].append(eigenvectors[0, 2])
    eigen_dict['eigenvec_3_2'].append(eigenvectors[1, 2])
    eigen_dict['eigenvec_3_3'].append(eigenvectors[2, 2])
    # Store eigenvalues
    eigen_dict['eigenval_1'].append(eigenvalues[0])
    eigen_dict['eigenval_2'].append(eigenvalues[1])
    eigen_dict['eigenval_3'].append(eigenvalues[2])

# Finally add generated features to the DataFrame
df_aggregate = pd.concat([df_aggregate, pd.DataFrame(eigen_dict)], axis=1)

In [ ]:
df_aggregate

In [ ]:
# Add user and class information to the new data set after the feature extraction
df_aggregate['User'] = df_user
df_aggregate['Class'] = df_class

**Evaluation Strategy**:<br>
We will use a k-Fold cross-validation to evaluate our model <br>
We randomly choose 2 User for the test set <br>
Then we remove the users from the selectable list so that each user is in the test data set at most once over all k runs <br>
For the next split we again choose 2 random User, and so on<br>

In [ ]:
import random

def get_train_test_user(df:pd.DataFrame, user_list:list, num_user_test:int=2):
    '''This function returns the indices for the training and test set.
    The function randomly selects two users for the test set and the remaining
    users for the training set.
    
    return: train_indices, test_indices'''

    # Create a list of indices for the training and test set
    train_indices = []
    test_indices = []
    
    # Generate 2 random numbers between 0 and 14
    test_user_1, test_user_2, test_user_3 = random.sample(user_list, num_user_test)
    print(f'User picked for test set: {test_user_1}, {test_user_2}')
    

    # Iterate over the users
    for user in user_list:
        # Get the indices for the current user
        indices = df[df['User'] == user].index
        # Append the indices to the list
        if user == test_user_1 or user == test_user_2 or user == test_user_3:
            test_indices.extend(indices)
        else:
            train_indices.extend(indices)
    
    # Remove the test users from the user list so they cannot be selected again
    user_list = [x for x in user_list if x != test_user_1 and x != test_user_2]

    return train_indices, test_indices, user_list

def custom_cv_approach(df:pd.DataFrame, user_list:list, num_user_test:int=2):
    '''
    each user is iteratively left out from training and used as a test set. 
    We then tests the generalization of the algorithm to new users. 
    A 'User' attribute is provided to accomodate this strategy. 
    '''
    def cv_ratio(y_test, df):
        print(f'Ratio of test set: {len(y_test)/len(df)}')
    
    # Get the indices for the training and test set
    train_indices, test_indices, user_list = get_train_test_user(df, user_list, num_user_test)
    # Create the training and test set
    X_train = df.iloc[train_indices, :]
    X_train = X_train.sample(frac=1, random_state=2023).reset_index(drop=True) #Shuffle
    y_train = X_train.pop('Class')
    X_train.pop('User')
    X_test = df.iloc[test_indices, :]
    X_test = X_test.sample(frac=1, random_state=2023).reset_index(drop=True) #Shuffle 
    y_test = X_test.pop('Class')
    X_test.pop('User')

    # Print the ratio of the test set
    cv_ratio(y_test, df)

    return X_train, y_train, X_test, y_test, user_list



In [ ]:
user_group = df_aggregate.groupby(['User'], sort=False)
user_group.count()

In [ ]:
#TODO add augmentation to the data set.
# e.g. add Jitter or Shuffle data

#### b. Dimensionality reduction

Of course we will also keep the raw data set and a data set with extracted features. In a later step we want to compare different data sets and their perfomance.<br>
Here we will extract one data set that contains pca components

##### PCA

In [ ]:
# We will choose n_components because their explained variance ratio is above 0.95
n_components = 25
pca = PCA(n_components=n_components)
df_pca = df_raw.copy()

# Drop the user and class information, they are not needed for the PCA
df_user = df_pca.pop('User')
df_class = df_pca.pop('Class')

pca.fit(df_pca)
df_pca_features = pd.DataFrame(pca.transform(df_pca))

# Now add the user and class information again
df_pca_features['User'] = df_user
df_pca_features['Class'] = df_class

#### c.I Split data - Wrong way

#### Demonstration data set (WRONG WAY) - mixed user

******************************************************
DEMONSTRATION: this shows how NOT to do it:<br>
Splitting the naive way (similar data points will be in both sets)
******************************************************

In [ ]:
df_raw

In [ ]:

# Split the data into train and test set
X_train_mixed, X_test_mixed, y_train_mixed, y_test_mixed = train_test_split(df_raw, df_raw['Class'], test_size=0.25)
user_group = df_raw.groupby(['User'], sort=False)

We normalize the data using a Min-Max-Scaler

In [ ]:
# Normalize the data with MinMaxScaler

# Create the scaler
scaler = MinMaxScaler()

# Fit the scaler to the training data
scaler.fit(X_train_mixed)

# Transform the training and test data
X_train_mixed = scaler.transform(X_train_mixed)
X_test_mixed = scaler.transform(X_test_mixed)

#### c.II Split data properly


We now have a CV loop:
I split the data in the same loop, where I train the model for a specific k-fold <br>
Therefore I dont split the data beforehand in this section

# 1. Testing Phase I: Baseline Models

After we have trained all the models, we want to compare the model performance using selected metrics. Therefore we need to save the metrics

In [ ]:
df_results = pd.DataFrame(columns=['model', 'Accuracy', 'Precision', 'Recall', 'F1_score'])

First, we write a training function to have a common interface for all conventional machine learning models.

In [ ]:
def train_model(df:pd.DataFrame, model_func: Callable, scaler_choice:str, kwargs_dict:dict):
    '''This function provides a common interface to train the classic ml models.
    In a loop we iterate over different train and test sets and train the model.
    We leave out k users from the data set and use them as test set.
    '''

    # Create a list of users
    user_list = [0, 1, 2, 5, 6, 8, 9, 10, 11, 12, 13, 14]

    # Number of users to be used for the test set
    num_user_test = 3
    num_of_iterations = 4

    best_model = None
    best_acc = 0
    X_test_best = None
    y_test_best = None

    # Save evaluation metrics in a dictionary
    eval_dict = {'train': {'accuracy':[], 'precision':[], 'recall':[], 'f1': []},
                    'test': {'accuracy':[], 'precision':[], 'recall':[], 'f1': []}}

    for i in range(num_of_iterations):

        print('*'*50)
        print(f'CV Run: {i}')

        model = model_func(**kwargs_dict)

        # Split the data
        X_train, y_train, X_test, y_test, user_list = custom_cv_approach(df, user_list, num_user_test=num_user_test)

        if scaler_choice == 'normalize':
            # Normalize the data
            scaler = MinMaxScaler()
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
        elif scaler_choice == 'standard':
            # Standard Scale the data
            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
        else:
            print('No scaling applied')

        # Fit the model
        model.fit(X_train, y_train)
        model_acc = model.score(X_test, y_test)
        
        # Print the results
        print('Train accuracy: ', model.score(X_train, y_train))
        print('(CV-) Test accuracy: ', model_acc)

        # Save the results
        y_pred_train = model.predict(X_train)
        train_acc = accuracy_score(y_train, y_pred_train)
        train_precision = precision_score(y_train, y_pred_train, average='macro')
        train_recall = recall_score(y_train, y_pred_train, average='macro')
        f1_score_train = f1_score(y_train, y_pred_train, average='macro')
        eval_dict = save_to_eval_dict(eval_dict, 'train', train_acc, train_precision, train_recall, f1_score_train)

        y_pred_test = model.predict(X_test)
        val_acc = accuracy_score(y_test, y_pred_test)
        val_precision = precision_score(y_test, y_pred_test, average='macro')
        val_recall = recall_score(y_test, y_pred_test, average='macro')
        f1_score_test = f1_score(y_test, y_pred_test, average='macro')

        eval_dict = save_to_eval_dict(eval_dict, 'test', val_acc, val_precision, val_recall, f1_score_test)

        # Save best model
        if model_acc > best_acc:
            best_model = model
            best_acc = model_acc
            X_test_best = X_test
            y_test_best = y_test

    return best_model, X_test_best, y_test_best, eval_dict


In [ ]:
def grid_search(df:pd.DataFrame, model_func: Callable, scaler_choice:str, cv_k:int, kwargs_dict:dict):
    '''
    Grid search function for conventional ML models.
    similiar to the train_model function.

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe containing the data, either raw or aggregated.
    model_func : Callable
        Function that returns a model object.
    scaler_choice : str
        Choice of scaler to be used. Either 'normalize' or 'standard'.
    cv_k : int
        Number of iterations for the cross validation.
    '''

    # Create a list of users
    user_list = [0, 1, 2, 5, 6, 8, 9, 10, 11, 12, 13, 14]

    # Number of users to be used for the test set
    num_user_test = 3

    best_model = None
    best_acc = 0
    best_param = None
    X_test_best = None
    y_test_best = None

    # Save all grid search objects in a list
    grid_search_list = []

    for i in range(cv_k):

        print('*'*50)
        print(f'CV Run: {i}')

        # Split the data
        X_train, y_train, X_test, y_test, user_list = custom_cv_approach(df, user_list, num_user_test=num_user_test)

        if scaler_choice == 'normalize':
            # Normalize the data
            scaler = MinMaxScaler()
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
        elif scaler_choice == 'standard':
            # Standard Scale the data
            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
        else:
            print('No scaling applied')

        # IMPORTANT: Define own train test split (otherwise GridSearch uses 5-CrossValidation)
        X_complete = np.concatenate((X_train, X_test), axis=0)
        y_complete = np.concatenate((y_train, y_test), axis=0)
        # The indices which have the value -1 will be kept in train.
        # The indices which have zero or positive values, will be kept in test
        test_fold = np.concatenate([np.full(len(X_train), -1, dtype=np.int8), np.zeros(len(X_test), dtype=np.int8)])
        cv_train_test = PredefinedSplit(test_fold)
        # Check how many splits will be done, based on test_fold
        print('Number of splits: ',cv_train_test.get_n_splits())

        # Create grid search object
        grid_search = GridSearchCV(cv=cv_train_test, **kwargs_dict)
        # Alternativce a Random Search
        # rnd_search = RandomizedSearchCV(svm_model, n_jobs=n_jobs, param_distributions=params, n_iter=num_iter, cv=cv_train_test, verbose=3, retrain=False)

        # Fit the model
        grid_search.fit(X_complete, y_complete)
    
        grid_search_list.append(grid_search)

        print(f"Best score: {round(grid_search.best_score_, 3)}")
        print(f"Best params: {grid_search.best_params_}")

        # Save best model
        if grid_search.best_score_ > best_acc:
            best_acc = grid_search.best_score_
            X_train_best = X_train
            y_train_best = y_train
            X_test_best = X_test
            y_test_best = y_test
            best_param = grid_search.best_params_
    
    # Retrain the best model
    # Usually we can use the refit parameter of the search object
    # but the would also fit on the test data
    model = model_func(**best_param)
    model.fit(X_train_best, y_train_best)
    
    return model, X_test_best, y_test_best, grid_search_list


## Random Forest Classifier

In [ ]:
# Hyperparameters:
#You need to check model descriptions for the hyperparameters. 
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest#sklearn.ensemble.RandomForestClassifier
#-----------------------------------------------------------------
# Number of trees in the forest:
n_estimators = 100 # default
# Number of features to consider when looking for the best split:
max_features = 'sqrt' # 'auto' is deprecated
# Maximum depth of the tree:
max_depth = None
# Minimum number of samples required to split an internal node:
min_samples_split = 2
# Minimum number of samples required to be at a leaf node:
min_samples_leaf = 1
# Grow trees with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. 
max_leaf_nodes = None
# Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree:
bootstrap = False
# Whether to use out-of-bag samples to estimate the generalization score. Only available if bootstrap=True.
oob_score = False
# Number of jobs to run in parallel. (-1) means use all.
n_jobs = -1
# Random state
random_state = 2023
#-----------------------------------------------------------------


In [ ]:
def plot_feature_importance_rf(rfc_model:RandomForestClassifier, feature_names:list):
    '''
    This function plots the feature importance of the random forest classifier.

    Parameters:
        rfc_model: Random Forest Classifier model
        X_train: Training data set, only for the feature names

    '''

    feature_importance = np.array(rfc_model.feature_importances_)
    feature_names = np.array(feature_names)

    data={'Feature names':feature_names,'Feature importance':feature_importance}
    fi_df = pd.DataFrame(data)
    fi_df.sort_values(by=['Feature importance'], ascending=False,inplace=True)

    plt.clf()
    plt.figure(figsize=(10,8))
    sns.barplot(x=fi_df['Feature importance'], y=fi_df['Feature names'])
    plt.title('Random Forest Feature Importance')
    plt.xlabel('Feature Importance')

    plt.show()

### a) Mixed dataset (wrong way)

In [ ]:
# Creating the classifier:
RFC_demo = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, \
                              max_leaf_nodes=max_leaf_nodes, bootstrap=bootstrap,oob_score=oob_score, n_jobs=n_jobs, random_state=random_state)
RFC_demo.fit(X_train_mixed, y_train_mixed)
RFC_demo.score(X_test_mixed, y_test_mixed)

In [ ]:
mixed_data_report = Report('rfc_mixed', RFC_demo, X_test_mixed, y_test_mixed, description=['mixed_data', 'raw features'])
print(mixed_data_report)
precision_recall_multiclass(RFC_demo, X_test_mixed, y_test_mixed)
y_pred_mixed = RFC_demo.predict(X_test_mixed)
plot_cm(y_test_mixed, y_pred_mixed)

Too good to be true...<br>
This was a demonstration how we should not split the data set!
Many people on kaggel etc. made this mistake

From now on we will not use this (wrong) processed data set

### b) Custom CV - with raw data

In [ ]:
# Hyperparameters:
#Random Forest raw features
rf_raw_hyperparams = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'max_leaf_nodes': max_leaf_nodes,
                'bootstrap': bootstrap,
                'oob_score': oob_score,
                'n_jobs': n_jobs,
                'random_state': random_state}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_raw, RandomForestClassifier, 'normalize', rf_raw_hyperparams)

#### Evaluation

In [ ]:
raw_w_cv_data_report = Report('rfc_raw', best_model, X_test_best, y_test_best, description=['cv_data', 'raw features'])
df_results = raw_w_cv_data_report.get_report_as_df(df_results)
print(raw_w_cv_data_report)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred_split_raw = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred_split_raw)

Here we can see that its difficult for the model to classify the "Fist" and "Point2" data points

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

In [ ]:
feature_list = df_raw.columns.to_list()
feature_list.remove('User')
feature_list.remove('Class')
plot_feature_importance_rf(best_model, feature_list)

### c) Custom CV - with extracted features

In [ ]:
# From now on we will use the following data
df_aggregate

In [ ]:
# Hyperparameters:
#Random Forest raw features
rf_extract_hyperparams = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'max_leaf_nodes': max_leaf_nodes,
                'bootstrap': bootstrap,
                'oob_score': oob_score,
                'n_jobs': n_jobs,
                'random_state': random_state}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_aggregate, RandomForestClassifier, 'normalize', rf_extract_hyperparams)

#### Evaluation

In [ ]:
extract_w_cv_data_report = Report('rfc_extract', best_model, X_test_best, y_test_best, description=['cv_data', 'extracted features'])
df_results = extract_w_cv_data_report.get_report_as_df(df_results)
print(extract_w_cv_data_report)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred_cv_extract = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred_cv_extract)

With the extracted features the model improved in predicting "Point2" data points, but the perfomance for predicting "Grab" decreased

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

In [ ]:
feature_list = df_aggregate.columns.to_list()
feature_list.remove('User')
feature_list.remove('Class')
plot_feature_importance_rf(best_model, feature_list)

### d) Custom CV - with pca components

In [ ]:
# Hyperparameters:
#Random Forest pca features
rf_pca_hyperparams = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'max_leaf_nodes': max_leaf_nodes,
                'bootstrap': bootstrap,
                'oob_score': oob_score,
                'n_jobs': n_jobs,
                'random_state': random_state}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_pca_features , RandomForestClassifier, 'normalize', rf_pca_hyperparams)

In [ ]:
pca_w_cv_data_report = Report('rfc_pca', best_model, X_test_best, y_test_best, description=['cv_data', 'pca features'])
df_results = pca_w_cv_data_report.get_report_as_df(df_results)
print(pca_w_cv_data_report)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred_cv_pca = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred_cv_pca)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

In [ ]:
feature_list = df_pca_features.columns.to_list()
feature_list.remove('User')
feature_list.remove('Class')
plot_feature_importance_rf(best_model, feature_list)

## Logistic Regression

In [ ]:
# Hyperparameters:
penalty = 'l2'
C = 1.0 #regularization strength. The smaller the value, the stronger the regularization.
random_state = 2023
solver = 'lbfgs' # One of the possible solver for multiclass problems (‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’)
max_iter = 300

### a) Custom CV - with raw data

In [ ]:
# Hyperparameters:
#Logistic regression raw features
lg_raw_hyperparams = {'penalty': penalty,
                'C': C,
                'random_state': random_state,
                'solver': solver,
                'max_iter': max_iter}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_raw, LogisticRegression, 'standard', lg_raw_hyperparams)

Error occured:
ConvergenceWarning: lbfgs failed to converge (status=1)
-> increase max_iter

#### Evaluation

In [ ]:
raw_w_cv_data_report_lg = Report('lg_raw', best_model, X_test_best, y_test_best, description=['LogReg', 'cv_data', 'raw features'])
df_results = raw_w_cv_data_report_lg.get_report_as_df(df_results)
print(raw_w_cv_data_report_lg)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

### b) Custom CV - with extracted features

In [ ]:
# Hyperparameters:
#Logistic regression extract features
lg_extract_hyperparams = {'penalty': penalty,
                'C': C,
                'random_state': random_state,
                'solver': solver,
                'max_iter': max_iter}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_aggregate, LogisticRegression, 'standard', lg_extract_hyperparams)

#### Evaluation

In [ ]:
extract_w_cv_data_report_lg = Report('lg_extract', best_model, X_test_best, y_test_best, description=['LogReg', 'cv_data', 'extracted features'])
df_results = extract_w_cv_data_report_lg.get_report_as_df(df_results)
print(extract_w_cv_data_report_lg)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

## Support Vector Machine (SVM)

### a) Custom CV - with raw data

In [ ]:
# Hyperparameters:
#SVC_raw
svc_raw_hyperparams = {
    'C': 1.0, #regularization strength. The smaller the value, the stronger the regularization.
    'gamma':'scale',
    'kernel': 'rbf',
    'decision_function_shape': 'ovo',
    'random_state': 2023,
    'max_iter': -1, # -1 means no limit
    'cache_size': 200, # in MB
    'probability': True # needed for predict_proba later on
}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_raw, SVC, 'standard', svc_raw_hyperparams)

#### Evaluation

In [ ]:
raw_w_cv_data_report_svm = Report('svm_raw', best_model, X_test_best, y_test_best, description=['SVM', 'cv_data', 'raw features'])
df_results = raw_w_cv_data_report_svm.get_report_as_df(df_results)
print(raw_w_cv_data_report_svm)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

### b) Custom CV - with extracted features

In [ ]:
# Hyperparameters:
#SVC_aggregate
svc_extract_hyperparams = {
    'C': 1.0, #regularization strength. The smaller the value, the stronger the regularization.
    'gamma':'scale',
    'kernel': 'rbf',
    'decision_function_shape': 'ovo',
    'random_state': 2023,
    'max_iter': -1, # -1 means no limit
    'cache_size': 200, # in MB
    'probability': True # needed for predict_proba later on
}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_aggregate, SVC, 'standard', svc_extract_hyperparams)

#### Evaluation

In [ ]:
extract_w_cv_data_report_svm = Report('svm_extract', best_model, X_test_best, y_test_best, description=['SVM', 'cv_data', 'extracted features'])
df_results = extract_w_cv_data_report_svm.get_report_as_df(df_results)
print(extract_w_cv_data_report_svm)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

### c) Custom CV - with pca features

In [ ]:
# Hyperparameters:
#SVC_pca
svc_pca_hyperparams = {
    'C': 1.0, #regularization strength. The smaller the value, the stronger the regularization.
    'gamma':'scale',
    'kernel': 'rbf',
    'decision_function_shape': 'ovo',
    'random_state': 2023,
    'max_iter': -1, # -1 means no limit
    'cache_size': 200, # in MB
    'probability': True # needed for predict_proba later on
}

best_model, X_test_best, y_test_best, eval_dict = train_model(df_pca_features, SVC, 'standard', svc_pca_hyperparams)

#### Evaluation

In [ ]:
pca_w_cv_data_report_svm = Report('svm_pca', best_model, X_test_best, y_test_best, description=['SVM', 'cv_data', 'pca features'])
df_results = pca_w_cv_data_report_svm.get_report_as_df(df_results)
print(pca_w_cv_data_report_svm)
precision_recall_multiclass(best_model, X_test_best, y_test_best)
y_pred_pca_svm = best_model.predict(X_test_best)
plot_cm(y_test_best, y_pred_pca_svm)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

### Grid Search - with extracted data and raw data

#### I. Grid Search with extracted features

In [ ]:
# We want to run a GridSearch without internal CV!
# We will run our own CV-Split because we have to split by User!
# Have a look at this thread If you are interested: https://stackoverflow.com/questions/44636370/scikit-learn-gridsearchcv-without-cross-validation-unsupervised-learning/44682305#44682305
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001],
              'kernel':['linear','rbf']} # alternative for C space: use np.logspace

static_params = {'decision_function_shape': 'ovo',
                    'random_state': 2023,
                    'max_iter': -1, # -1 means no limit
                    'cache_size': 200, # in MB
                    'probability': True # needed for predict_proba later on
}

# Add static params to param_grid
#param_grid.update(static_params)

svc_extract_search = {'estimator': SVC(**static_params),
                      'param_grid': param_grid,
                      'refit' : False, #We later want to refit only on X_train, y_train
                      'scoring': 'accuracy',
                      'verbose': 2,
                      'n_jobs': -1
}
cv_k = 3 # Number CV folds
model, X_test_best, y_test_best, grid_search_list = grid_search(df_aggregate, SVC, 'standard', cv_k, svc_extract_search)

TODO other models to try <br>
LightGBM

In [ ]:
def plot_grid_search(grid_search_list, grid_param_1, grid_param_2):
    
    for idx, grid_search in enumerate(grid_search_list):
        print('Grid Search #', idx)
        data = grid_search.cv_results_
    
        # extract relevant data
        params = [d[grid_param_1] for d in data['params']]
        param2_values = sorted(list(set([d[grid_param_2] for d in data['params']])))
        scores = data['mean_test_score']#np.array([d['mean_test_score'] for d in data])

        # create figure
        fig, ax = plt.subplots()

        # plot scores for each value of param2
        for i, param2_value in enumerate(param2_values):
            x = [params[j] for j in range(len(params)) if data['params'][j][grid_param_2] == param2_value]
            y = [scores[j] for j in range(len(params)) if data['params'][j][grid_param_2] == param2_value]

            # Get all indices with the same x value:
            # get unique values and their counts
            unique_values, value_counts = np.unique(x, return_counts=True)
            # loop through unique values and get indices of elements with that value
            indices_by_value = []
            for value in unique_values:
                indices_by_value.append(np.where(x == value)[0])
            x = unique_values
            # Calculate the mean of y values that have the same x value
            y = [np.mean([y[i] for i in indices_by_value[k]]) for k in range(len(unique_values))]
            
            ax.plot(x, y, label=f'{grid_param_2}={param2_value}')

        # set axis labels and legend
        ax.set_xlabel(grid_param_1)
        ax.set_ylabel('Mean Test Score')
        ax.legend(title=grid_param_2)

        plt.show()

In [ ]:
plot_grid_search(grid_search_list, 'C', 'kernel')
plot_grid_search(grid_search_list, 'C', 'gamma')

#### II. Grid Search with raw features

We want to search in the same parameter space that we used for grid search with the extracted features. Therefore we simply have to call the grid_search function with the raw dataset

In [ ]:
model, X_test_best, y_test_best, grid_search_list = grid_search(df_raw, SVC, 'standard', cv_k, svc_extract_search)

In [ ]:
plot_grid_search(grid_search_list, 'C', 'kernel')
plot_grid_search(grid_search_list, 'C', 'gamma')

In [ ]:
#TODO boxplot of the results
'''
ax = sns.boxplot(data = f2_df, linewidth=1, showfliers=False)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
sns.set(rc = {'figure.figsize':(8,10)})
ax.set(ylabel='F2-Score')
ax.set_title('F2-Score Deviation of different Models')
'''

## Conclusion Testing Phase I

In [ ]:
def plot_model_comparison(df_results):
    '''
    Plot a comparison of the models based on the evaluation metrics
    '''

    df = df_results.copy()
    # Set the model as the index
    df.set_index('model', inplace=True)
    # Create a 2x2 grid of subplots
    fig, axs = plt.subplots(2, 2, figsize=(8, 6))
    # Create color list
    colors = ['tab:blue', 'tab:red', 'tab:green', 'tab:orange', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']


    # Plot each metric in a subplot
    for i, metric in enumerate(df.columns):
        row = i // 2
        col = i % 2
        axs[row, col].bar(df.index, df[metric], color=colors[:len(df.index)], width=0.3)
        axs[row, col].set_xticklabels(df.index.to_list(), rotation=90)
        axs[row, col].set_ylabel(metric)
        axs[row, col].set_title(f'{metric} Comparison')
        axs[row, col].set_ylim([0, 1])

    # Adjust the spacing between subplots
    plt.tight_layout()
    # Display the plots
    plt.show()


In [ ]:
df_results
plot_model_comparison(df_results)
best_score = df_results.iloc[df_results['Accuracy'].idxmax()]
print('**Model with highest accuracy score**')
print(best_score)

TODO

Extracted features?
* good, we see an improvement ...

Confusion Matrix:
* what cases are difficult
* which are easy
* explanation why

Feature reduction
* good/bad
* PCA
* which models got worses

Best model:
* e.g. rf with extracted data

# 4. Testing Phase II: Model Developement

Help Section:<br>
You have problems that the kernel freezes mulitple times while training? The Training process seems to run in the background and hence there will appear new models (if you save them as pickle files).
But there is no information that the training process finished (running forever)
follow this thread:<br>
https://stackoverflow.com/questions/52261597/keras-model-fit-verbose-formatting <br>
and install:
pip install ipywidgets

## Base MLP structure

* Activation Functions
    - output layer: one neuron for each class (5)
    - we want the the probability of the each class, -> **softmax**
    - hidden Layer: start with **ReLU**

* Optimizer
    - **rmsprop**

* loss
    - **categorical_crossentropy**

* Hidden Layers and Number of Neurons
    - start with small architecture, increase size

* Metric
    - **accuracy**
    - **precission**


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


def build_mlp_model(name:str, hyperparams:dict, input_shape: tuple, output_shape: int) -> keras.Sequential:
    'Build MLP classification network'

    if hyperparams['hidden_layer'] != len(hyperparams['units_hidden_layer']):
        raise ValueError('Number of hidden layers and values provided for number of units per hidden layer do not match')
    
    # Use l1, l2 regularization?
    if hyperparams['weight regularisation l1'] and hyperparams['weight regularisation l2']:
        kernel_regu = keras.regularizers.L1L2(l1=hyperparams['weight regularisation l1'],
                                              l2=hyperparams['weight regularisation l2'])
    elif hyperparams['weight regularisation l1']:
        kernel_regu = keras.regularizers.L1(l1=hyperparams['weight regularisation l1'])
    elif hyperparams['weight regularisation l2']:
        kernel_regu = keras.regularizers.L2(l2=hyperparams['weight regularisation l2'])
    else: # Do not use regularization
        kernel_regu = None

    model = keras.Sequential(name=name)
    model.add(keras.Input(shape=input_shape))

    # Hidden Layer
    for num_hl in range(hyperparams['hidden_layer']):
        model.add(layers.Dense(hyperparams['units_hidden_layer'][num_hl],
                               activation=hyperparams['activation_hidden'],
                               kernel_initializer=hyperparams['kernel_initializer'],
                               bias_initializer=hyperparams['bias_initializer'],
                               kernel_regularizer=kernel_regu
                               ))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(rate=hyperparams['dropout']))

    model.add(layers.Dense(output_shape, activation='softmax'))

    model.compile(optimizer=hyperparams['optimizer'], loss=hyperparams['loss'], metrics=hyperparams['metrics'])

    return model

In [ ]:
def plot_learning_curves(hist, parameters, name:str):
    
    plt.plot(hist['epoch'][:],hist['loss'][:], "k--", linewidth=1.5, label="Training")
    plt.plot(hist['epoch'][:],hist['val_loss'][:], "b-.", linewidth=1.5, label="CV test")
    plt.legend()
    plt.ylim(0,max(hist['loss'][:].max(), hist['val_loss'][:].max())+0.2)
    plt.xlabel("Epochs"),  plt.ylabel("categorical_crossentropy")

    plt.title(f'Learning Curve: {name}', fontsize=18)
    plt.show()

In [ ]:
def plot_multiple_learning_curves(name, hyperparams):
    """
    Plot the learning curves for each cv run
    """

    # Load hist of every cv run

    all_hist = []

    for i in range(4):
        hist = load_history(f'{name}_cv{i}')
        all_hist.append(hist)
        
    # Plot the learning curves for each cv run
    for i in range(4):
        plot_learning_curves(all_hist[i], hyperparams, f'{name}_cv{i}')


In [ ]:
def plot_accuracy_curves(hist, parameters, name:str):
    '''
    Plot the accuracy curves of the training and test set
    '''
    
    plt.plot(hist['epoch'][:],hist['categorical_accuracy'][:], "k--", linewidth=1.5, label="Training")
    plt.plot(hist['epoch'][:],hist['val_categorical_accuracy'][:], "b-.", linewidth=1.5, label="CV test")
    plt.legend()
    plt.ylim(0,1.0)
    plt.xlabel("Epochs"),  plt.ylabel("categorical_accuracy")

    plt.title(f'Accuracy Curve: {name}', fontsize=18)
    plt.show()

In [ ]:
def plot_multiple_acc_curves(name, hyperparams):
    """
    Plot the accuracy curves for each cv run
    """

    # Load hist of every cv run

    all_hist = []

    for i in range(4):
        hist = load_history(f'{name}_cv{i}')
        all_hist.append(hist)
        
    # Plot the accuracy curves for each cv run
    for i in range(4):
        plot_accuracy_curves(all_hist[i], hyperparams, f'{name}_cv{i}')

In [ ]:
def save_history(hist, name):

    # Get path to save the history
    hist_folder = 'model/single_run/history'
    model_path = join(hist_folder, name)
    
    with open(model_path, 'wb') as file_pi:
        pickle.dump(hist, file_pi)

def load_history(name):

    # Get path to save the history
    hist_folder = 'model/single_run/history'
    model_path = join(hist_folder, name)
    
    with open(model_path, 'rb') as file_pi:
        history = pickle.load(file_pi)
    
    return history

In [ ]:
def load_model(name):

    # Get path to save the history
    model_folder = 'model/single_run'
    model_path = join(model_folder, name)
    
    model = keras.models.load_model(model_path)
    
    return model

In [ ]:
def list_used_parameters(parameters:dict):
    print('Used parameters:')
    for parameter, value in parameters.items():
        print(f'{parameter}: {value}')

In [ ]:
def decode_one_hot(one_hot_encoded):
    return np.argmax(one_hot_encoded)

In [ ]:
def calc_f1_score_mlp(model, X, y):
    # Calculate f1 score of mlp
    y_pred1 = model.predict(X)
    y_pred = np.argmax(y_pred1, axis=1)

    # to one hot encoding
    y_pred = to_categorical(y_pred)

    return f1_score(y, y_pred , average="macro")

In [ ]:
def train_k_mlp_model(name, df:pd.DataFrame, hyperparams:dict):
    '''
    Train a MLP model with k-fold cross validation (leave n users out strategy)

    Parameters
    ----------
    name : str, Name of the model.

    Returns
    ----------
    best_model : keras.Sequential, The best model.
    best_acc : float, The best accuracy.
    X_test_best : numpy.ndarray, The best test set.
    y_test_best : numpy.ndarray, The best test set.
    '''
    

    # Create a list of users
    user_list = [0, 1, 2, 5, 6, 8, 9, 10, 11, 12, 13, 14]

    # Number of users to be used for the test set
    num_user_test = 3
    num_of_iterations = 4

    best_model = None
    best_acc = 0
    X_test_best = None
    y_test_best = None

    all_acc_val = []
    all_prec_val = []

    # Evaluation dict
    eval_dict = {'train': {'accuracy':[], 'precision':[], 'recall':[], 'f1': []},
                 'test': {'accuracy':[], 'precision':[], 'recall':[], 'f1': []}}

    describe_model = True

    for i in range(num_of_iterations):

        print('*'*50)
        print(f'CV Run: {i}')

        # Split the data #TODO rename to X_train etc. because function also use by raw and later pca
        X_train_cv_extract, y_train_cv_extract, X_test_cv_extract, y_test_cv_extract, user_list = custom_cv_approach(df, user_list, num_user_test=num_user_test)

        # Normalize the data
        scaler = StandardScaler()
        scaler.fit(X_train_cv_extract)
        X_train_cv_extract = scaler.transform(X_train_cv_extract)
        X_test_cv_extract = scaler.transform(X_test_cv_extract)
        # Befor one hot encoding, class has to start at 0
        y_train_cv_extract = y_train_cv_extract-1
        y_test_cv_extract = y_test_cv_extract-1
        # y label to one hot encode
        y_train_cv_extract = to_categorical(y_train_cv_extract)
        y_test_cv_extract = to_categorical(y_test_cv_extract)

        # Build the model
        # TODO
        mlp_model = build_mlp_model(name, hyperparams, input_shape=X_train_cv_extract.shape[1:], output_shape=y_train_cv_extract.shape[1])

        # Only print the model summary once
        if describe_model:
            mlp_model.summary()
            describe_model = False

        # Create callback
        # Use early stopping later because it is a form of regularization
        #early_stop_callback = keras.callbacks.EarlyStopping(monitor="val_categorical_accuracy", min_delta=1e-4, patience=5, verbose=1)
        checkpoint_callback = keras.callbacks.ModelCheckpoint(f'model/single_run/{name}_cv{i}.h5', save_best_only=True)
        
        callbacks = [TqdmCallback(verbose=0), checkpoint_callback]
        if hyperparams['reduce_lr']:
            reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.2,
                                           patience=5, min_lr=0.00001,
                                           min_delta=0.001)
            callbacks.append(reduce_lr_callback)

        # Train the model
        history = mlp_model.fit(X_train_cv_extract, 
                                y_train_cv_extract, 
                                epochs=hyperparams['num_epochs'], 
                                batch_size= hyperparams['batch_size'],
                                validation_data=(X_test_cv_extract, y_test_cv_extract),
                                verbose=0,
                                shuffle=True,
                                callbacks=[TqdmCallback(verbose=0), checkpoint_callback])

        hist = pd.DataFrame(history.history)
        hist['epoch'] = history.epoch

        # Save the history
        save_history(hist, f'{name}_cv{i}')

        #Evaluating the training performance:
        train_loss, train_acc, train_precision, train_recall = mlp_model.evaluate(x=X_train_cv_extract, y=y_train_cv_extract, batch_size=hyperparams['batch_size'], verbose=3)
        f1_score_train = calc_f1_score_mlp(mlp_model, X_train_cv_extract, y_train_cv_extract)

        eval_dict = save_to_eval_dict(eval_dict, 'train', train_acc, train_precision, train_recall, f1_score_train)
        print('-'*70)
        print('Evaluation of training Data: \n', 'training loss: ', train_loss, 'training accuracy: ', train_acc)

        #Evaluating the CV pperformance:
        val_loss, val_acc, val_precision, val_recall = mlp_model.evaluate(x=X_test_cv_extract, y=y_test_cv_extract, batch_size=hyperparams['batch_size'], verbose=0)
        f1_score_test = calc_f1_score_mlp(mlp_model, X_test_cv_extract, y_test_cv_extract)
        eval_dict = save_to_eval_dict(eval_dict, 'test', val_acc, val_precision, val_recall, f1_score_test)
        all_acc_val.append(val_acc)
        all_prec_val.append(val_precision)
        print('Evaluation of validation Data: \n', 'cv loss: ', val_loss, 'cv accuracy: ', val_acc)

        # Save best model
        if val_acc > best_acc:
            best_model = mlp_model
            best_acc = val_acc
            X_test_best = X_test_cv_extract
            y_test_best = y_test_cv_extract

    #Lets see the overall score as average of the scores of all the folds:
    print('-'*70)
    print('(all CV runs combined)')
    print('Mean Accuracy  for the validation dataset: ', np.mean(all_acc_val))
    print('Mean Precision for the validation dataset: ', np.mean(all_prec_val))
    print('-'*70)

    # Save the model
    best_model.save(f'model/single_run/{name}_best_model.h5')

    # Save the evaluation dict
    save_eval_dict_pkl(eval_dict, f'{name}_eval_dict')

    # Plot the learning curves
    plot_learning_curves(hist, hyperparams, name)

    return best_model, best_acc, X_test_best, y_test_best, eval_dict


### Methodology

<ins>Disclaimer</ins>: If you are only interested in the results, you can skip this section! I would like to guarantee a red thread here and show that the models did not fall from the sky like this

Since I adjusted hyperparameters several times in the following sections, I would like to give the reader a brief overview here of how the adjustment steps went:

1. Model seems to overfit from the very beginning <br>
I used a structure like this:<br>
number of neurons in the hidden layers: [32, 64, 64, 32, 16]<br>
<img src="img/overfit_big_model.png" width=330 height=310 /> <br>
I decided to reduce the number neurons and layers. The new structure looked like this:<br>
number of neurons in the hidden layers: [16, 16, 8]<br>
<img src="img/overfit_small_model.png" width=330 height=310 /> <br>
I added Batch normalization and drop out layer to prevent the model from overfitting. <br>
Problem still seemed to be presence. So I reduced the network complexity even further:<br>
number of neurons in the hidden layers: [8, 8, 8]<br>
<img src="img/underfit.png" width=330 height=310 /> <br>
Now the model was underfitting. The training loss was not decreasing that much after a few epochs. The validation loss was on a same level. Another thing I was wondering about was the weird behaviour of the validation loss. Right from the beginning of training it was increasing and the curve has fluctuated very strongly.

2. Validation loss is more fluctuating then the training loss<br>
After some internet reasearch I got inspiration what to look for. I have added more validation data as it may not be representative of the data. Now I selected three user for the validation data instead of instead of two.
3. Check preprocessing:
Some posts also suggested two have a look into my preprocessing steps. Some people suggested I might have processed the validation and the training data in a different way
I did not shuffle the data!
```python
model.fit(..., shuffle=True) #Keras model
```
-> only shuffels the training set!! not the the validation set

4. First real baseline model
After decreasing the learning rate and increased the drop out rate for the drop out layer, I got a quiet descent MLP model: <br>
<img src="img/baseline_model.png" width=330 height=310 /> <br>


### a) Custom CV - with raw data

In [ ]:
# Hyperparameters
hyperparams = {'num_epochs': 35, # 35,
               'batch_size': 32,
               'hidden_layer': 3,
               'units_hidden_layer': [16, 16, 8],#[64, 64, 32, 16],
               'activation_hidden': keras.layers.LeakyReLU(alpha=0.1), #'LeakyReLU' #'relu',
               'activation_output': 'softmax',
               'loss': 'categorical_crossentropy',
               'metrics': ['categorical_accuracy', 'Precision', tf.keras.metrics.Recall()],
               'optimizer': keras.optimizers.legacy.Adam(learning_rate=0.00005), #0.0003
               'kernel_initializer': 'he_normal', # used for ReLU activation function
               'bias_initializer': 'zeros',
               'initialization': '-',
               'weight regularisation l1': '',
               'weight regularisation l2': '',
               'dropout': 0.35,  # typically between 0.3 and 0.5 (half of weights get 0)
               'early Stopping': 'False',
               'reduce_lr': False} 

To get an Idea How long the model trains: <br>
22min for 4 runs
* each trains for 35 epochs

Highly depends on the used Hardware! <br>
Here we use an M1 macbook with GPU tf vesion

In [ ]:
name = 'mlp_base_model_raw'
df = df_raw
best_model, best_acc, X_test_best, y_test_best, eval_dict = train_k_mlp_model(name, df, hyperparams)

In [ ]:
plot_multiple_learning_curves(name, {'test': 'test'})

In [ ]:
#adjust to mlp, KeyError: 'accuracy'
data_report_mlp_base_raw = Report('mlp_raw', best_model, X_test_best, y_test_best, description=['MLP base', 'cv_data', 'raw features'])
#df_results = data_report_mlp_base_raw.get_report_as_df(df_results)
print(data_report_mlp_base_raw)

# Prepare data to be compatible with confusion matrix function
y_pred = best_model.predict(X_test_best)
y_pred = np.argmax(y_pred, axis=1)+1
y_test_int = np.argmax(y_test_best, axis=1)+1
plot_cm(y_test_int, y_pred)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

In [ ]:
run_mode = 'retrain' # 'retrain' or 'load_model'
if run_mode == 'load_model':
    hist = load_history('mlp_base_model_raw_cv3')
    plot_learning_curves(hist, hyperparams, 'test')
    eval_dict = load_eval_dict_pkl('mlp_base_model_raw_eval_dict')
    plot_eval_dict_barplot_new(eval_dict)

First I tried a model with 32 16 8 units in the hidden layer<br>
the validation los increased rigth from the beginning while the test loss was decreasing, which is a sign of overfitting<br>
insert pic 

### b) Custom CV - with extracted features

#### Training

In [ ]:
# I figured out the the model trained with extracted features need more epochs
hyperparams['num_epochs'] = 60

In [ ]:
name = 'mlp_base_model_extracted'
df = df_aggregate
best_model, best_acc, X_test_best, y_test_best, eval_dict = train_k_mlp_model(name, df, hyperparams)

In [ ]:
data_report_mlp_base_extract = Report('mlp_extract', best_model, X_test_best, y_test_best, description=['MLP base', 'cv_data', 'extract features'])
#df_results = data_report_mlp_base_raw.get_report_as_df(df_results)
print(data_report_mlp_base_extract)

# Prepare data to be compatible with confusion matrix function
y_pred = best_model.predict(X_test_best)
y_pred = np.argmax(y_pred, axis=1)+1
y_test_int = np.argmax(y_test_best, axis=1)+1
plot_cm(y_test_int, y_pred)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

In [ ]:
plot_multiple_learning_curves(name, {'test': 'test'})

In [ ]:
plot_multiple_acc_curves(name, {'test': 'test'})

#### Evaluation

In [ ]:
#TODO save hist in list or load hist of every model and plot learning curve of every cv model
# retrain model on whole data (validation + train) leave one user out as test set (instead of best_model)

In [ ]:
data_report_mlp_base_raw = Report('mlp_extract', best_model, X_test_best, y_test_best, description=['MLP base', 'cv_data', 'extracted features'])
#df_results = data_report_mlp_base_raw.get_report_as_df(df_results)
print(data_report_mlp_base_raw)

# Prepare data to be compatible with confusion matrix function
y_pred = best_model.predict(X_test_best)
y_pred = np.argmax(y_pred, axis=1)+1
y_test_int = np.argmax(y_test_best, axis=1)+1
plot_cm(y_test_int, y_pred)

In [ ]:
plot_eval_dict_barplot_new(eval_dict)

In [ ]:
# RESULTS
# Now we have base mlp model that we trained in a stable way (no big fluctuations etc.) and which is also not overfitting
# This should be a good starting point for a hyperparameter search and fine tuning of regularization technique

### c) Custom CV -with pca data

In [ ]:
name = 'mlp_base_model_pca'
df = df_pca_features
best_model, best_acc, X_test_best, y_test_best, eval_dict = train_k_mlp_model(name, df, hyperparams)

In [ ]:
plot_multiple_learning_curves(name, {'test': 'test'})

### Results

Already training the baseline MLP models has required several iterative fits. Partial regularization methods were already used, e.g. dropout layer. In this section, however, these were only applied to be able to train the model correctly at all.

(In section 3 more regularization methods are used and fine tuned to achieve the best model performance).

## PointNet

Note: for this archtitecture we have to use the original, raw data because we need a Point Cloud!

From the Paper: "PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation" (Charles R. Qi et al)
https://arxiv.org/abs/1612.00593

DISCLAIMER:
Implementation with Keras:<br>
https://keras.io/examples/vision/pointnet/

Here we replicate the network architecture published in the original paper with the help of this blogpost!

General
* Point cloud =  a geometric data structure with irregular format
* paper proposes a novel neural network called PointNet
* PointNet: directly consumes point clouds + respects the permutation invariance of points in the input
* unified architecture for object classification, part segmentation, and scene semantic parsing
* simple, efficient, and effective

Architecture
* deal with unordered input set: use of a single symmetric function, max pooling
* network learns a set of optimization functions/criteria that select interesting or informative points of the point cloud and encode the reason for their selection
* final fully connected layers: aggregate these learnt optimal values into the global descriptor for the entire shape (shape classification) or are used to predict per point labels (shape segmentation).

Each convolution and fully-connected layer (with exception for end layers) consits of 
* Convolution / Dense
* Batch Normalization
* ReLU Activation.

In [ ]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

PointNet consists of two core components
* primary MLP network
* transformer net (T-net)
    * aims to learn an affine transformation matrix by its own mini network
    * used twice:
        * 1.to transform the input features (n, 3) into a canonical representation
        * 2.affine transformation for alignment in feature space (n, 3)

What will we do?
* implement main network 
* drop the t-net mini models as layers in the graph

In [ ]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))
    
    def get_config(self): #TODO required for saving model
        return {'test': 'test'}


In [ ]:
def tnet(inputs, num_features):
    '''Build T-net layers'''
    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 8) # Original 32 in paper
    x = conv_bn(x, 32) # Original 64 in paper
    x = conv_bn(x, 64) # Original 512 in paper
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 32) # Original 256 in paper
    x = dense_bn(x, 8) # Original 128 in paper
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [ ]:
def build_pointnet(num_points, num_classes):
    '''Use the functional API to build a PointNet model (different from the Sequential API)'''
    inputs = keras.Input(shape=(num_points, 3))

    x = tnet(inputs, 3)
    x = conv_bn(x, 16) #OG 32
    x = conv_bn(x, 16) #OG 32
    x = tnet(x, 16) #OG 32
    x = conv_bn(x, 16) #OG 32
    x = conv_bn(x, 16) #OG 64
    x = conv_bn(x, 32) #OG 512
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 64) #OG 256
    x = layers.Dropout(0.3)(x)
    x = dense_bn(x, 32) #OG 128
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")

    return model

Now we have all utility functions we need for our PointNet model<br>
We have to preprocess the input data to be compatible with the network

In [ ]:
def custom_cv_approach_point_clouds(np_batches:np.array, df:pd.DataFrame, user_list:list, num_user_test:int=2) -> Tuple[np.array, np.array, np.array, np.array, list]:
    '''
    MODIFIED for point cloud data
    each user is iteratively left out from training and used as a test set. 
    We then tests the generalization of the algorithm to new users. 
    A 'User' attribute is provided to accomodate this strategy. 
    '''

    def cv_ratio(y_test, df):
        print(f'Ratio of test set: {len(y_test)/len(df)}')
    
    # Get the indices for the training and test set
    train_indices, test_indices, user_list = get_train_test_user(df, user_list, num_user_test)
    # Shuffle data
    shuffle(train_indices)
    shuffle(test_indices)
    print(f'train_indices examples: {len(train_indices)}')
    print(f'test_indices examples: {len(test_indices)}')
    # Create the training and test set
    # MODIFIED for point cloud data
    X_train = np_batches[train_indices, :, :]
    y_train = df.iloc[train_indices, :]['Class']
    X_test = np_batches[test_indices, :, :]
    y_test = df.iloc[test_indices, :]['Class']

    cv_ratio(y_test, df)

    return X_train, y_train, X_test, y_test, user_list

In [ ]:
def train_k_pn_model(name, np_batches:np.array, df:pd.DataFrame, hyperparams:dict):
    '''
    Train multiple PointNet models with k-fold cross validation (leave n users out strategy)

    Parameters
    ----------
    name : str, Name of the model.
    np_batches : numpy.ndarray, The data.
    df : pandas.DataFrame, We will use this dataframe ONLY to get the labels.
    hyperparams : dict, The hyperparameters.

    Returns
    ----------
    best_model : keras.Sequential, The best model.
    best_acc : float, The best accuracy.
    X_test_best : numpy.ndarray, X_test set
    y_test_best : numpy.ndarray, y_test set
    '''
    

    # Create a list of users
    user_list = [0, 1, 2, 5, 6, 8, 9, 10, 11, 12, 13, 14]

    # Number of users to be used for the test set
    num_user_test = 3
    num_of_iterations = 4

    best_model = None
    best_acc = 0
    X_test_best = None
    y_test_best = None

    all_acc_val = []
    all_prec_val = []

    # Evaluation dict
    #TODO add recall and f1 score
    eval_dict = {'train': {'accuracy':[], 'precision':[], 'recall':[], 'f1': []},
                 'test': {'accuracy':[], 'precision':[], 'recall':[], 'f1': []}}

    describe_model = True

    for i in range(num_of_iterations):

        print('*'*50)
        print(f'CV Run: {i}')

        # Split the data into train and test set
        X_train_cv_pn, y_train_cv_pn, X_test_cv_pn, y_test_cv_pn, user_list = custom_cv_approach_point_clouds(np_batches, df_raw, user_list, num_user_test=num_user_test)

        # Normalize the data
        scaler = MinMaxScaler()
        X_train_cv_pn = scaler.fit_transform(X_train_cv_pn.reshape(-1, X_train_cv_pn.shape[-1])).reshape(X_train_cv_pn.shape)
        X_test_cv_pn = scaler.transform(X_test_cv_pn.reshape(-1, X_test_cv_pn.shape[-1])).reshape(X_test_cv_pn.shape)
        # Before one hot encoding, class has to start at 0
        y_train_cv_pn = y_train_cv_pn-1
        y_test_cv_pn = y_test_cv_pn-1
        # y label to one hot encode
        y_train_cv_pn = to_categorical(y_train_cv_pn)
        y_test_cv_pn = to_categorical(y_test_cv_pn)

        # Build model
        print('X_train.shape: ', X_train_cv_pn.shape)
        print('Sample input shape: ', X_train_cv_pn[0].shape)
        print('Sample input: ', X_train_cv_pn[0])
        print('y_train.shape: ', y_train_cv_pn.shape)
        
        max_num_points = 10
        #TODO add hyperparams
        pointnet_model = build_pointnet(max_num_points, y_train_cv_pn.shape[1])

        # Only print the model summary once
        if describe_model:
            pointnet_model.summary()
            describe_model = False

        # Create callback
        #early_stop_callback = keras.callbacks.EarlyStopping(monitor="val_categorical_accuracy", min_delta=1e-4, patience=5, verbose=1)
        checkpoint_callback = keras.callbacks.ModelCheckpoint(f'model/single_run/{name}_cv{i}.h5', save_best_only=True)
        
        pointnet_model.compile(
            loss=hyperparams['loss'],
            optimizer=hyperparams['optimizer'],
            metrics=hyperparams['metrics'],
            )

        # Train the model
        history = pointnet_model.fit(X_train_cv_pn, 
                                y_train_cv_pn, 
                                epochs=hyperparams['num_epochs'], 
                                batch_size= hyperparams['batch_size'],
                                validation_data=(X_test_cv_pn, y_test_cv_pn),
                                verbose=0,
                                shuffle=True,
                                callbacks=[TqdmCallback(verbose=0), checkpoint_callback])

        hist = pd.DataFrame(history.history)
        hist['epoch'] = history.epoch

        # Save the history
        save_history(hist, f'{name}_cv{i}')

        #Evaluating the training performance:
        train_loss, train_acc, train_precision, train_recall = pointnet_model.evaluate(x=X_train_cv_pn, y=y_train_cv_pn, batch_size=hyperparams['batch_size'], verbose=1)
        f1_score_train = calc_f1_score_mlp(pointnet_model, X_train_cv_pn, y_train_cv_pn)
        eval_dict = save_to_eval_dict(eval_dict, 'train', train_acc, train_precision, train_recall, f1_score_train)
        print('-'*70)
        print('Evaluation of training Data: \n', 'training loss: ', train_loss, 'training accuracy: ', train_acc)

        #Evaluating the CV pperformance:
        val_loss, val_acc, val_precision, val_recall = pointnet_model.evaluate(x=X_test_cv_pn, y=y_test_cv_pn, batch_size=hyperparams['batch_size'], verbose=1)
        f1_score_test = calc_f1_score_mlp(pointnet_model, X_test_cv_pn, y_test_cv_pn)
        eval_dict = save_to_eval_dict(eval_dict, 'train', val_acc, val_precision, val_recall, f1_score_test)
        all_acc_val.append(val_acc)
        all_prec_val.append(val_precision)
        print('Evaluation of validation Data: \n', 'cv loss: ', val_loss, 'cv accuracy: ', val_acc)

        # Save best model
        if val_acc > best_acc:
            best_model = pointnet_model
            best_acc = val_acc
            X_test_best = X_test_cv_pn
            y_test_best = y_test_cv_pn

    #Lets see the overall score as average of the scores of all the folds:
    print('-'*70)
    print('(all CV runs combined)')
    print('Mean Accuracy  for the validation dataset: ', np.mean(all_acc_val))
    print('Mean Precision for the validation dataset: ', np.mean(all_prec_val))
    print('-'*70)

    # Save the model
    best_model.save(f'model/single_run/{name}_best_model.h5')

    # Save the evaluation dict
    save_eval_dict_pkl(eval_dict, f'{name}_eval_dict')

    # Plot the learning curves
    plot_learning_curves(hist, hyperparams, name)

    return best_model, best_acc, X_test_best, y_test_best, eval_dict


In [ ]:
# Each row of data (multiple data points) got stored as single dataframe where each row is one single data point
# Convert the dataframe to 3D numpy Matrix 
np_batches = np.array(list(map(pd.DataFrame.to_numpy, concat_batches)))
np_batches.shape

In [ ]:
# We need to reshape the data to be able to normalize it
# EXAMPLE: for 2 batches (for the real data we will do it in the training loop)
print(np_batches[0:2].shape)
reshaped = np_batches[0:2].reshape(-1, np_batches[0].shape[-1])
print(reshaped.shape)

In [ ]:
# Hyperparameters
#TODO set number of epochs to 15
# TODO have a look what "sparse_categorical_crossentropy" is
hyperparams = {'num_epochs': 10,
               'batch_size':10,
               'hidden_layer':2,
               'units_hidden_layer': 32,
               'activation_hidden': 'relu',
               'activation_output': 'softmax',
               'loss': 'categorical_crossentropy',
               'metrics': ['categorical_accuracy', 'Precision', tf.keras.metrics.Recall()],
               'optimizer': keras.optimizers.legacy.Adam(learning_rate=0.00005), #OG 0.001
               'initialization': '-',
               'weight regularisation l2': '-',
               'dropout': '-',  # typically between 0.3 and 0.5 (half of weights get 0)
               'early Stopping': 'False'} 

In [ ]:
#TODO have a look what is point 0 X0, Y0

In [ ]:
# Run the model
name = 'pointnet_base'
best_model, best_acc, X_test_best, y_test_best, eval_dict_pn = train_k_pn_model(name, np_batches, df_raw, hyperparams)

In [ ]:
plot_multiple_learning_curves(name, {'test': 'test'})

In [ ]:
eval_dict_pn

### Conclusion Testing Phase II

In [ ]:
df_results
plot_model_comparison(df_results)
best_score = df_results.iloc[df_results['Accuracy'].idxmax()]
print('**Model with highest accuracy score**')
print(best_score)

# 5. Testing Phase III: Model Regularization and Hyperparameter optimization

In this section we try to fine tune the model to achieve a better performance. We have already partially applied regularization methods in "Testing Phase II", but they were only used there to merely stabilize the training process. It would probably have been possible to achieve a stable training process ("good" loss curve) without regularization. But the bug fixxing was more an iterative process in which different error sources etc. should be excluded.

In this section we focus on the fine tuning of these regularization methods.


In a first step we will tune the model by hand. This is followed by a randomized search

we will focus on following hyperparameter
* number of neurons
* regularization
* number of layers
* activation function
* dropout
* Initialization
* Optimizer
* Batch Size
* Epochs

## Manual Parameter Tuning

Keeping in mind that the model with extracted features achieved a higher perfomance we will focus on this data


In this section we start using EarlyStopping and LearningRateScheduler


In [ ]:
# Hyperparameters
hyperparams = {'num_epochs': 55,
               'batch_size': 32,
               'hidden_layer': 3,
               'units_hidden_layer': [16, 16, 8],#[64, 64, 32, 16],
               'activation_hidden': keras.layers.LeakyReLU(alpha=0.1), #'LeakyReLU' #'relu',
               'activation_output': 'softmax',
               'loss': 'categorical_crossentropy',
               'metrics': ['categorical_accuracy', 'Precision', tf.keras.metrics.Recall()],
               'optimizer': keras.optimizers.SGD(lr=0.001, momentum=0.9) #keras.optimizers.legacy.Adam(learning_rate=0.0001), #0.0003
               'kernel_initializer': 'he_normal', # used for ReLU activation function
               'bias_initializer': 'zeros',
               'initialization': '-',
               'weight regularisation l1': '',
               'weight regularisation l2': 0.001,
               'dropout': 0.5,  # typically between 0.3 and 0.5 (half of weights get 0)
               'early Stopping': 'False',
               'reduce_lr': True} 

In [ ]:
#TODO run with hiher l2 norm 18.03

In [ ]:
name = 'mlp_base_model_extracted_tuned'
df = df_aggregate
best_model, best_acc, X_test_best, y_test_best, eval_dict = train_k_mlp_model(name, df, hyperparams)

In [ ]:
plot_multiple_learning_curves(name, {'test': 'test'})

In [ ]:
plot_multiple_acc_curves(name, {'test': 'test'})

### Results

Manual fine tuning of the model is a highly iterative process. Instead of cluttering the entire notebook with multiple trial runs and making it confusing, I will describe the iterative process here. The actual changes were modified directly in the hyperparameters.

For example, we see here that the learning rate is too low. One could reach the goal with a low learning rate, but this wastes computing resources unnecessarily, because we need more epochs.
I decide to increase the learning rate

![title](img/lr_too_low.png)

I also figured out that reducing the learning rate after some time can help the model to train

In [ ]:
hist = load_history('impact_lr_scheduler')
plot_learning_curves(hist, {'None': None}, 'test')

In [ ]:
# insert impact_lr_scheduler.h5 learning curve

# reducing the learning rate after some time can help the model

#### Best model

In [ ]:
# TODO plot best_model_extract

In [ ]:
# Used hyperparamter
'''
# Hyperparameters
hyperparams = {'num_epochs': 50,
               'batch_size': 32,
               'hidden_layer': 3,
               'units_hidden_layer': [16, 16, 8],#[64, 64, 32, 16],
               'activation_hidden': keras.layers.LeakyReLU(alpha=0.1), #'LeakyReLU' #'relu',
               'activation_output': 'softmax',
               'loss': 'categorical_crossentropy',
               'metrics': ['categorical_accuracy', 'Precision', tf.keras.metrics.Recall()],
               'optimizer': keras.optimizers.legacy.Adam(learning_rate=0.0001), #0.0003
               'kernel_initializer': 'he_normal', # used for ReLU activation function
               'bias_initializer': 'zeros',
               'initialization': '-',
               'weight regularisation l1': '',
               'weight regularisation l2': 0.001,
               'dropout': 0.5,  # typically between 0.3 and 0.5 (half of weights get 0)
               'early Stopping': 'False',
               'reduce_lr': True} 
'''

loss=0.554, categorical_accuracy=0.786, precision=0.844, recall_2=0.722, val_loss=0.431, val_categorical_accuracy=0.897, val_precision=0.898, val_recall_2=0.896]

## Automatic hyperparameter search

To find a good value range for a specific hyperparamter, simply have a look online <br>
e.g. https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/
for L2 values

In [ ]:
from keras_tuner import HyperModel

class Classification_MLP_tuner(HyperModel):
    '''
    Build a HyperParameter Model with variable hyperparameters
    e.g. Optimizer, learning rate 
    
    Note: model.compile must be included in this function
    '''

    def __init__(self, name:str, input_shape, num_output, learning_rate:list, max_num_of_hidd_layer:int, max_num_of_neuron_per_layer:int, min_num_of_neuron_per_layer:int, activation_func_hidden_layer:list, weight_regularisation_l1:list, weight_regularisation_l2:list):
        self.name = name
        self.input_shape = input_shape
        self.num_ouput = num_output
        self.learning_rate = learning_rate
        self.max_num_of_hidd_layer = max_num_of_hidd_layer
        self.max_num_of_neuron_per_layer = max_num_of_neuron_per_layer
        self.min_num_of_neuron_per_layer = min_num_of_neuron_per_layer
        self.activation_func_hidden_layer = activation_func_hidden_layer
        self.weight_regularisation_l1 = weight_regularisation_l1
        self.weight_regularisation_l2 = weight_regularisation_l2

    def build(self, hp):

        model = keras.Sequential(name=self.name)
        model.add(layers.InputLayer(input_shape=self.input_shape))
        
        l1 = hp.Float('l1',
                 min_value=self.weight_regularisation_l1[0],
                 max_value=self.weight_regularisation_l1[1],
                 sampling='LOG',
                default=0)
        
        l2 = hp.Float('l2',
                 min_value=self.weight_regularisation_l2[0],
                 max_value=self.weight_regularisation_l2[1],
                 sampling='LOG',
                default=0)

        # Use l1, l2 regularization?
        if l1 and l2:
            kernel_regu = keras.regularizers.L1L2(l1=l1, l2=l2)
        elif l1:
            kernel_regu = keras.regularizers.L1(l1=l1)
        elif l2:
            kernel_regu = keras.regularizers.L2(l2=l2)
        else: # Do not use regularization
            kernel_regu = None
            
        # Choose activation function
        hp_activation = hp.Choice('dense_activation', values=self.activation_func_hidden_layer)
        if hp_activation == 'leaky_relu': # Leaky Relu does not have an alias in keras and the values list for choice can onl contain one type
            hp_activation = keras.layers.LeakyReLU(alpha=0.1)
            
        for i in range(hp.Int('num_layers', 2, self.max_num_of_hidd_layer)):
            model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=self.min_num_of_neuron_per_layer,
                                                max_value=self.max_num_of_neuron_per_layer,
                                                step=2),
                                    activation=hp_activation,
                                    kernel_initializer='he_normal',
                                    bias_initializer='zeros',
                                    kernel_regularizer=kernel_regu ))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(rate=hp.Choice(
                                            'dropout_rate',
                                            values=[0.3, 0.4, 0.5]
                                            )))
        model.add(layers.Dense(5, activation='softmax')) #TODO output variable

        #Compile
        opt = keras.optimizers.Adam(hp.Float(
                                'learning_rate',
                                min_value=self.learning_rate[0],
                                max_value=self.learning_rate[1],
                                sampling='LOG',
                                default=1e-3)
                                )
        metrics = [keras.metrics.CategoricalAccuracy()]
        loss_func = keras.losses.CategoricalCrossentropy()
        model.compile(loss=loss_func, optimizer=opt, metrics=metrics)
        
        return model

In [ ]:
def mlp_hyperparameter_search(df, max_trials:int, epochs:int, batch_size:int, load_search:Optional[str]=None):
    '''
    Suche nach den besten Hyperparametern des MLP
    Parameters
    ----------
    max_trials :
        maximum number of random models to investigate
    epochs :
        number of iterations to train neural network
    batch_size :
        Anzahl der Samples fuer die Gradient berechnet wird (somit gemittelt)
    '''

    # Create a list of users
    user_list = [0, 1, 2, 5, 6, 8, 9, 10, 11, 12, 13, 14]

    # Number of users to be used for the test set
    num_user_test = 3
    
    #create folder
    search_path = join('model/hp_search', 'mlp')

    #Convert input data
    X_train, y_train, X_test, y_test, user_list = custom_cv_approach(df, user_list, num_user_test=num_user_test)

    # Normalize the data
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    # Before one hot encoding, class has to start at 0
    y_train = y_train-1
    y_test = y_test-1
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    #Transform to Dataset
    X_train_stream_labeled = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    X_test_stream_labeled = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    # Shuffle Data
    X_train_stream_labeled = X_train_stream_labeled.shuffle(buffer_size=df.shape[0])
    X_test_stream_labeled = X_test_stream_labeled.shuffle(buffer_size=df.shape[0])
    #create batches
    X_train_stream_labeled = X_train_stream_labeled.batch(batch_size)
    X_test_stream_labeled = X_test_stream_labeled.batch(batch_size)

    #build model
    input_shape = (X_train.shape[1], )
    output_shape = len(np.unique(y_train))
    
    #TODO write them in hyper param dict
    hyperparams_search = {'activation_func_hidden_layer': ['relu', 'leaky_relu', 'elu'], #['tanh', 'sigmoid', 'elu', 'relu']
                       'min_num_of_neuron_per_layer': 2,
                       'max_num_of_neuron_per_layer': 16,
                       'max_num_of_hidd_layer': 5,
                       'learning_rate': [5e-5, 5e-3],
                          'weight_regularisation_l1': [0.000001, 0.1],
                          'weight_regularisation_l2': [0.000001, 0.1]
    }
    
    
    mlp_model = Classification_MLP_tuner('mlp', input_shape,
                                         output_shape, learning_rate=hyperparams_search['learning_rate'],
                                         max_num_of_hidd_layer=hyperparams_search['max_num_of_hidd_layer'], 
                                         max_num_of_neuron_per_layer=hyperparams_search['max_num_of_neuron_per_layer'], 
                                         min_num_of_neuron_per_layer=hyperparams_search['min_num_of_neuron_per_layer'], 
                                         activation_func_hidden_layer=hyperparams_search['activation_func_hidden_layer'],
                                        weight_regularisation_l1 = hyperparams_search['weight_regularisation_l1'],
                                        weight_regularisation_l2 = hyperparams_search['weight_regularisation_l2'])
    
    #REGULARIZATION
    early_stop_callback = keras.callbacks.EarlyStopping(monitor="val_categorical_accuracy", min_delta=1e-3, patience=5, verbose=1)
    #reduce_lr_callback = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.2,
    #                                       patience=5, min_lr=0.00001,
    #                                       min_delta=0.001)
    
    callbacks = [early_stop_callback]

    # Create name with timestamp
    t = time.localtime()
    timestamp = time.strftime('%b-%d-%Y_%H%M', t)

    #train
    if load_search is None:
        tuner_search = keras_tuner.RandomSearch(mlp_model, objective='val_categorical_accuracy', max_trials=max_trials, project_name=join(search_path, f'prj_{timestamp}_mlp'))
        tuner_search.search(X_train_stream_labeled, 
                            epochs=epochs, 
                            validation_data=X_test_stream_labeled,
                            callbacks=callbacks)

        # Not required anymore, search can be easily restored from search path!
        '''
        pickle_path = join(search_path, f'mlp_search_{timestamp}.pkl')
        with open(pickle_path, 'wb') as handle:
            pickle.dump(tuner_search, handle)
        '''
    else:
        tuner_search = keras_tuner.RandomSearch(mlp_model, objective='val_categorical_accuracy', max_trials=max_trials, overwrite=False, project_name=load_search)
        return tuner_search




There are different hyperparameter we want to analyze the impact of:
* Number of layers
* Number of nodes
* Activation function
* Optimizer
* Learning rate
* Number of epochs -> we will use an Early stopping Callback
* Batch size
* Adding weight regularization
* Adding dropout

Due to the amount of hyperparameter combination:<br>
a univariate hyperparemeter search is not target-oriented, because the parameters influence each other.
we have a multivariate problem. The number of possible value combinations, however, cannot (at least with the hardware available to me) all be examined. A possible approach is to randomly select parameter sets, i.e. to perform a random search.

Procedure:
* for number of examined hyperparmeter sets (e.g. here 150)
    * Define hyperparameter space (some parameters continuous, others discrete)
    * draw random combination of values
    * for #CV_folds:
        * select n users for validation set
        * use rest for training with the randomly drawn hyperparameter combination

Finally, analyze which hyperparmeter combination has the best validation accuracy

First perform a random search in hyperparameter space with the raw data

In [ ]:
#TOOD write all hyperparameter options in one dict

Reference time: 3h 3min to try 150 models

In [ ]:
mlp_hyperparameter_search(df_raw, max_trials=150, epochs=35, batch_size=32)

We should always make sure that we can restore or load important evvaluation data, whole models or search results.<br>
Coming back after 2 days of random hyperparameter search only to find out that the kernel crashed or restarted at any point in time is suboptimal<br>
Simply relying on the fact that the jupyter kernel will store the python objects is not good

In [ ]:
project_path = '/home/josh/dde1_hand_motion/model/hp_search/mlp/prj_Mar-09-2023_2332_mlp_raw' #'/home/josh/dde1_hand_motion/model/hp_search/mlp/prj_Mar-16-2023_1415_mlp'
load_model = True

if load_model:
    print('*'*50)
    print('Reload tuner search')
    tuner_search = mlp_hyperparameter_search(df_raw, max_trials=1, epochs=1, batch_size=10, load_search = project_path)
    print(tuner_search.get_best_hyperparameters(4))
    print(tuner_search.results_summary(4))

Now perform the same search with the extracted data set

In [ ]:
mlp_hyperparameter_search(df_aggregate, max_trials=150, epochs=10, batch_size=10)

As I said before:<br>
always store the search object, in case we have to restore it after the kernel crashed. <br>
Here we can simply load the Search Object where we used the extracted features (even if we come back after several days)

In [ ]:
project_path = '/home/josh/dde1_hand_motion/model/hp_search/mlp/prj_Mar-11-2023_0019_mlp_extract'
load_model = False

if load_model:
    tuner_search = mlp_hyperparameter_search(df_raw, max_trials=1, epochs=1, batch_size=10, load_search = project_path)
    print(tuner_search.get_best_hyperparameters(4))
    print(tuner_search.results_summary(4))

### Evaluation

In [ ]:
def rand_search_info(pkl_path:str, num_of_model:int):

    if os.path.getsize(pkl_path) > 0:
        with open(pkl_path, 'rb') as f:
            rand_search = pickle.load(f)
    else:
        print('File is empty!')
        return

    print('best parameter:', rand_search.best_params_)
    print('best score:', rand_search.best_score_)

    search_df = pd.DataFrame(rand_search.cv_results_)
    search_df.set_index('rank_test_score', inplace=True)
    
    for model_rank in range(1, num_of_model+1):
        print(f'\n###{model_rank}_model###')

        try:
            params_n_model = search_df['params'][model_rank]
            score_n_model = search_df['split0_test_score'][model_rank]
        except KeyError:
            print(f'no model with rank {model_rank} (previous is multiple')
            continue
        print(params_n_model)
        print(score_n_model)
    
    return rand_search.best_score_, rand_search.best_params_


def keras_tuner_search_results(pkl_path:str):
    
    with open(pkl_path, 'rb') as f:
        rand_search = pickle.load(f)

    result_summary = rand_search.results_summary()
    print(result_summary)
    best_model = rand_search.get_best_models(num_models=1)[0]

    return best_model

In [ ]:
'''
import pickle
import os
search_path = 'model/hp_search/mlp/mlp_search_Mar-09-2023_2332.pkl'
keras_tuner_search_results(search_path)
rand_search_info(search_path, num_of_model=10)
'''

## PointNet

# 6. Evaluation of the model predictions

###  All Models/Overall Comparison

# 7. Lessons Learnt and Conclusions

tell us what you found and what you learned!

Logistic Regression
https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

Gradient Descent

Random Forests

Boosting (LightGBM)
https://lightgbm.readthedocs.io/en/latest/

## Model 1


### Training

### Evaluation

Confusion Matrix
PR Curve
ROC Curve

### Discussion

## Model 2


### Training

### Evaluation

### Discussion